In [1]:
import networkx as nx
import numpy as np
from collections import Counter, defaultdict
import random
from scipy.special import softmax
import torch


from main.utils import load_dataset, InverseProblemDataset, adj_process
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [2]:
dataset = 'power_grid' # 'cora_ml', 'citeseer', 'ms_academic', 'pubmed'
graph = load_dataset(dataset)
adj = graph.adj_matrix.todense()

In [3]:
inverse_pairs = InverseProblemDataset(dataset)

In [41]:
import pickle

with open('power_grid_SI.SG', 'rb') as f:
    graph = pickle.load(f)
    
adj, inverse_pairs, prob_matrix = graph['adj'].toarray(), graph['inverse_pairs'], graph['prob'].toarray()

In [42]:
train_set, test_set = torch.utils.data.random_split(inverse_pairs, [len(inverse_pairs)-8, 8])
data = test_set[:].numpy()
print(data.shape)

(8, 4941, 2)


In [43]:
G = nx.from_numpy_matrix(adj)

In [44]:
def NetSleuth(G, k, target_vector):
    
    N = len(G)
    G.remove_nodes_from([n for n in G if n not in np.where(target_vector==1)[0]])
    lap = nx.laplacian_matrix(G).toarray()
    
    seed = []
    loss = 1e3
    t = 0
    while len(seed) < k:
        t += 1
        # loss = loss_new
        value, vector = np.linalg.eig(lap)
        index = np.argmax(vector[np.argmin(value)])
        seed_index = list(G.nodes)[index]
        seed.append(seed_index)
        G.remove_node(seed_index)
        lap = nx.laplacian_matrix(G).toarray()
        # loss_new = score(G, seed, beta, k, t, N)
    
    return np.array(seed)

In [45]:
def write_to_gephi(dataset, gt_y, gt_x, pred_x):
    graph = load_dataset(dataset)
    true_G = nx.from_scipy_sparse_matrix(graph.adj_matrix)
    pred_G = nx.from_scipy_sparse_matrix(graph.adj_matrix)
    true_y = nx.from_scipy_sparse_matrix(graph.adj_matrix)
    
    for i in true_G.nodes:
        true_G.nodes[i]['class'] = gt_x[i]
        pred_G.nodes[i]['class'] = pred_x[i]
        true_y.nodes[i]['inf_rate'] = gt_y[i]
        
    nx.write_gexf(true_G, "{}_Netsleuth_real.gexf".format(dataset))
    nx.write_gexf(pred_G, "{}_Netsleuth_pred.gexf".format(dataset))
    nx.write_gexf(true_y, "{}_Netsleuth_y.gexf".format(dataset))

In [46]:
accuracy = 0
precision = 0
recall = 0
f1 = 0
auc = 0

for pair in data:
    G = nx.from_scipy_sparse_matrix(graph['adj'])
    seed, target = pair[:, 0], pair[:, 1]
# seed, target = data[0, :, 0], data[0, :, 1]
    k = len(seed[seed==1])

    target[target>0.9] = 1
    target[target != 1] = 0
    print(k, len(target[target==1]))
    pred_index = NetSleuth(G, k*1.2, target)
    pred_seed = np.zeros(adj.shape[0])
    pred_seed[pred_index] = 1

    accuracy += accuracy_score(seed, pred_seed)
    precision += precision_score(seed, pred_seed)
    recall += recall_score(seed, pred_seed)
    f1 += f1_score(seed, pred_seed)
    auc += roc_auc_score(seed, pred_seed)

494 1005
494 1029
494 938
494 996
494 971
494 1032
494 1041
494 955


In [47]:
print(accuracy/8, precision/8, recall/8, f1/8, auc/8)

0.9000202388180529 0.5 0.6002024291497976 0.5455381784728611 0.7667641333965763


In [29]:
pred_seed

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0.])

In [30]:
seed

array([0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)

In [31]:
import networkx as nx

In [48]:
write_to_gephi('power_grid', target, pred_seed, seed)

In [62]:
# # 生成数据
# G = nx.from_scipy_sparse_matrix(graph.adj_matrix)
# probs = [0.01, 0.05, 0.1, 0.15, 0.3, 0.5]
# graph_size = adj.shape[0]

# for (u, v) in G.edges():
#     G.edges[u,v]['weight'] = random.choice(probs)

# # Getting the P matrix
# weighted_adj = nx.adjacency_matrix(G).toarray()
# for i in range(graph_size):
#     weighted_adj[i][np.where(weighted_adj[i]>0)] = softmax(weighted_adj[i][np.where(weighted_adj[i]>0)])

# # Controlling the seeds in the same region
# inverse_pairs = []
# for i in range(50):
#     _seeds = np.random.choice(80, 50, replace=False)
#     seed_vec = np.zeros((graph_size))
#     seed_vec[_seeds] = 1.0

#     inf_vec = np.zeros((graph_size))
#     inf_vec[_seeds] = 1.0

#     for x in range(10):
#         inf_vec=np.dot(inf_vec, weighted_adj)

#     inverse_pairs.append([seed_vec, inf_vec]
# inverse_pairs = np.array(inverse_pairs)

In [50]:
# 获取感染点的所有邻居，并计算邻居连接的感染点数目
def get_neighbor(G, seed):
    neighbor_list = []
    for s in seed:
        neighbor_list += [i for i in nx.all_neighbors(G, s)]
    result = Counter(neighbor_list)
    
    neighbor_count = defaultdict(list)
    for key, value in result.items():
        neighbor_count[value].append(key)
        
    return dict(neighbor_count)

In [4]:
# 定义二次项系数
def Cnk(n, k):
    if k==0: return 1
    if n==0: return 0
    
    return Cnk(n-1,k)+Cnk(n-1,k-1)

In [64]:
# # 计算得分L
# def score(G, seed, beta, k, t, N):
    
#     n = len(G)
    
#     n_seed = len(seed)
#     log_seed = np.log(n_seed)
    
#     # 计算L(S)
#     loss_s = 0
    
#     while log_seed >= 0:
#         loss_s += log_seed
#         log_seed = np.log(log_seed)
#     loss_s = loss_s + np.log(2.865064) + np.log(Cnk(N, n_seed))
    
#     neighbor_dict = get_neighbor(G, seed)
    
#     # 计算L(R|S)
#     loss_r = 0
#     log_t = np.log(t)
#     while log_t>= 0:
#         loss_r += log_t
#         log_t = np.log(log_t)
#     loss_r += np.log(2.865064)  

In [242]:
graph = load_dataset(dataset)
adj = graph.adj_matrix.todense()
G = nx.from_scipy_sparse_matrix(graph.adj_matrix)

In [245]:
print(accuracy, precision, recall)

0.9252669039145908 0.5588235294117647 0.5588235294117647
